# Übungsblatt 11 #

### Alice Ziegler, Daniel Schneider ###

## Aufgabe 11.1: Warenkorbanalyse ##

## Aufgabe 11.2: Warenkorbanalyse ##

### Einlesen des Datensatzes ###

In [1]:
import csv
from collections import Counter
import itertools

In [2]:
def readCsv(path,delimiter):
    with open(path, newline='') as csvfile:
        csvReader = csv.reader(csvfile, delimiter=delimiter)
        #for row in csvReader:
            #print(', '.join(row))
        return list(csvReader)

In [3]:
groceries = readCsv("data/groceries.csv",",")

In [4]:
Counter(x for subset in groceries for x in subset)

Counter({'Instant food products': 79,
         'UHT-milk': 329,
         'abrasive cleaner': 35,
         'artif. sweetener': 32,
         'baby cosmetics': 6,
         'baby food': 1,
         'bags': 4,
         'baking powder': 174,
         'bathroom cleaner': 27,
         'beef': 516,
         'berries': 327,
         'beverages': 256,
         'bottled beer': 792,
         'bottled water': 1087,
         'brandy': 41,
         'brown bread': 638,
         'butter': 545,
         'butter milk': 275,
         'cake bar': 130,
         'candles': 88,
         'candy': 294,
         'canned beer': 764,
         'canned fish': 148,
         'canned fruit': 32,
         'canned vegetables': 106,
         'cat food': 229,
         'cereals': 56,
         'chewing gum': 207,
         'chicken': 422,
         'chocolate': 488,
         'chocolate marshmallow': 89,
         'citrus fruit': 814,
         'cleaner': 50,
         'cling film/bags': 112,
         'cocoa drinks': 22,
         '

### a) Frequent Itemsets: a-priori-Algorithmus ###

In [5]:
def sortedTransaktions(D):
    Dsort = list(D)
    for i in range(len(Dsort)):
        Dsort[i] = sorted(Dsort[i], key=str.lower)
    return Dsort

In [6]:
def getUniverse(D):
    U = []
    for i in range(len(D)):
        for j in range(len(D[i])):
            if D[i][j] not in U:
                U.append(D[i][j])
    return sorted(U, key=str.lower)

In [7]:
def support(x,D):
    count = 0
    for j in range(len(D)):
        if isSubList(x,D[j]):
        #if all([(y in D[j]) for y in x]):
            count += 1
    return count/len(D)

In [8]:
def aPrioryGenerate(L, k):
    C = []
    
    for i in range(len(L)):
        for j in range(i,len(L)):
            if(L[i][0:k-1] == L[j][0:k-1]):
                c = list(L[i][0:k-1])
                c.extend(sorted([L[i][k-1],L[j][k-1]], key=str.lower))
                
                #if not c in C:
                if not hasInfrequentSubset(c, L):
                    C.append(c)
    
    return C

In [9]:
def hasInfrequentSubset(c, L):
    for i in range(len(c)):
        s = c[:i] + c[i+1:]
        
        for j in range(len(L)):
            if s == L[j]:
                break
        else:
            return True
    
    return False

In [10]:
def isSubList(subList,parentList):
    if len(subList) <= len(parentList):
        i = 0
        j = 0
        while j < len(parentList):
            if i >= len(subList):
                return True
            else:
                if subList[i] == parentList[j]:
                    i += 1
                    j += 1
                else:
                    j += 1
        
        if i >= len(subList):
            return True
    
    return False

In [11]:
def findFrequentItemsets(D, minSup):
    D = sortedTransaktions(D)
    
    L = []
    k = 0
    
    #Frequent-1-Itemsets
    C = [[u] for u in getUniverse(D)]
    #print(len(C))
    
    Lcurr = []
    for i in range(len(C)):
        if support(C[i],D) >= minSup:
            Lcurr.append(C[i])
    
    L.extend(Lcurr)
    #print(Lcurr)
    #print(len(Lcurr))
    
    while(Lcurr):
        k = k+1
        
        C = aPrioryGenerate(Lcurr, k)
        #print(len(C))
        
        Lcurr = []
        for i in range(len(C)):
            if support(C[i],D) >= minSup:
                Lcurr.append(C[i])
        
        #print(Lcurr)
        #print(len(Lcurr))
        
        L.extend(Lcurr)
        #L = [c for c in C if LPrev.count(c) >= minSup]
        
    return L

In [12]:
groceriesFrequent = findFrequentItemsets(groceries, 0.03)

#### Ergebnis: Frequent-Itemsets des `groceries`-Datensatzes ####

In [13]:
groceriesFrequent

[['beef'],
 ['berries'],
 ['bottled beer'],
 ['bottled water'],
 ['brown bread'],
 ['butter'],
 ['canned beer'],
 ['chicken'],
 ['chocolate'],
 ['citrus fruit'],
 ['coffee'],
 ['cream cheese '],
 ['curd'],
 ['dessert'],
 ['domestic eggs'],
 ['frankfurter'],
 ['frozen vegetables'],
 ['fruit/vegetable juice'],
 ['hamburger meat'],
 ['hygiene articles'],
 ['long life bakery product'],
 ['margarine'],
 ['napkins'],
 ['newspapers'],
 ['onions'],
 ['other vegetables'],
 ['pastry'],
 ['pip fruit'],
 ['pork'],
 ['rolls/buns'],
 ['root vegetables'],
 ['salty snack'],
 ['sausage'],
 ['shopping bags'],
 ['soda'],
 ['specialty chocolate'],
 ['sugar'],
 ['tropical fruit'],
 ['UHT-milk'],
 ['waffles'],
 ['whipped/sour cream'],
 ['white bread'],
 ['whole milk'],
 ['yogurt'],
 ['bottled water', 'whole milk'],
 ['citrus fruit', 'whole milk'],
 ['other vegetables', 'rolls/buns'],
 ['other vegetables', 'root vegetables'],
 ['other vegetables', 'soda'],
 ['other vegetables', 'tropical fruit'],
 ['other ve

### b) Confidence ###

In [14]:
def calculatePartitions(itemSet):
    rules = []
    combinations = []
    
    for size in range(1,len(itemSet)):
        combinations.extend([list(x) for x in itertools.combinations(itemSet,size)])
        
    #print(combinations)
    #print(len(combinations))
    
    for i in range(len(combinations)):
        for j in range(len(combinations)):
            if not (list(set(combinations[i]) & set(combinations[j]))):
                rules.append((combinations[i],combinations[j]))
    return rules

In [15]:
def findStrongRules(freqItemsets, D, minConfidence):
    D = sortedTransaktions(D)
    
    rules = []
    
    for itemSet in freqItemsets:
        rules.extend(calculatePartitions(itemSet))
    
    #print(rules)
    
    strongRules = []
    
    for (ruleL,ruleR) in rules:
        confidence = support(sorted(ruleL + ruleR, key=str.lower), D) / support(ruleL, D)
        #print(str(ruleL)+","+str(ruleR)+":"+str(confidence))
        
        if confidence >= minConfidence:
            strongRules.append((ruleL,ruleR))
    
    return strongRules

In [16]:
groceriesStrongRules = findStrongRules(groceriesFrequent,groceries,0.4)

#### Ergebnis: Starke Regeln des `groceries`-Datensatzes ####

In [17]:
for (ruleL,ruleR) in groceriesStrongRules:
    print(str(ruleL)+" => "+str(ruleR))

['root vegetables'] => ['other vegetables']
['root vegetables'] => ['whole milk']
['tropical fruit'] => ['whole milk']
['whipped/sour cream'] => ['whole milk']
['yogurt'] => ['whole milk']


## Aufgabe 11.3: Assoziationsregeln ##

Bei der Zuordnung der Elemente eines Itemsets zu einer Assoziationsregel gibt es grundsätzlich für jedes Element 3 Entscheidungsmöglichkeiten:
1. auf der linken Seite der Regel
2. gar nicht in der Regel
3. auf der rechten Seite der Regel

Das ergibt bei $d$ Elementen eine Anzahl von $A = 3^d$ Regeln.

Allerdings sind einige dieser Regeln nicht gültig. Die linke und rechte Regelseite dürfen nicht leer sein.

Man kann einen Entscheidungsbaum aufstellen, in dem jedes Element eine Ebene bildet, auf der für dieses Element die Entscheidung zwischen den oben genannten Möglichkeiten getroffen wird. (L steht in der folgenden Grafik für die Entscheidungsmöglichkeit für die linke Seite, R für die für die der rechten Seite und x für die Entscheidung das Objekt nicht in die Regel aufzunehmen)

![Baum](data/Baum.png "Entscheidungsbaum")

Dabei sind in diesem Baum nur die Pfade gültig, in denen mindestens einmal die Entscheidung für die linke Regelseite und mindestens einmal die Entscheidung für die rechte Regelseite getroffen wurde.

Das heißt umgekehrt, dass alle Pfade ungültig sind, in denen nur die Entscheidung für die immer gleiche Regelseite oder gar keine Regelseite getroffen wurde. In diesem Fall sind auf der letzten Ebene des Baums von den 3 möglichen Entscheidungen 2 ungültig. Mit jeder Ebene, um die der Baum wächst, verdoppelt sich die Anzahl dieser ungültigen Pfade, bei $d$ Ebenen des Baumes erhält man also $2^d$ ungültige Pfade. 

Daher kann man von den $A$ Assoziationsregeln $2^d$ ungültige Regeln abziehen.

Die folgende Grafik zeigt einen Teilbaum, bei dem bislang nur die linke Seite oder gar keine gewählt wurde. Gültige Pfade sind grün markiert, ungültige rot.

![BaumLinks](data/BaumLinks.png "Linker Pfad")

Dazu kommt der mittlere Teilbaum (Element nicht in der Regel), in dem es den Fall gibt, dass bis zur vorletzten Entscheidungsebene immer die Entscheidung für Nichtaufnahme in die Regel getroffen wurde. In diesem Fall sind sogar alle 3 Entscheidungen der letzten Ebene ungültig. Wenn der Baum um eine weitere Ebene wächst, entstehen drei Teilbäume, von denen wieder im mittleren alle 3 Entscheidungen ungültig sind. 

Im linken und rechten Teilbaum sind jeweils 2 von 3 Entscheidungen ungültig. Die Anzahl ungültiger Entscheidungen ist also von 3 auf 7 gewachsen. 

Man kann die ungültigen Entscheidungen auch als $x - 1$ mit $x:=4$ definieren. Dann gilt auch hier, dass sich mit jeder Ebene, die der Baum wächst, $x$ verdoppelt. Man kann also weitere $2^d - 1$ ungültige Regeln von $A$ abziehen.

Die folgende Grafik zeigt den Teilbaum, bei dem bislang immer die Entscheidung für keine Aufnahme in die Regel getroffen wurde. Gültige Pfade sind grün markiert, ungültige rot.

![BaumMitte](data/BaumMitte.png "Mittlerer Pfad")

Daraus ergibt sich insgesammt bei $d$ Elementen eines Itemsets also eine Anzahl von

$$
\begin {align}
& 3^d - 2^d - (2^d - 1) \\
=\quad & 3^d - 2*2^d + 1 \\
=\quad & 3^d - 2^{d+1} + 1 \\
\end{align}
$$
Assoziationsregeln.